In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from keras.optimizers import Nadam, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

import math, json

import pandas as pd
import numpy as np

import os.path

In [2]:
def import_log(filepath):
    df = pd.read_csv(filepath)
    return(df.values.tolist())

def remove_nan(lists):
    newlists = []
    for tr in lists:
        newlists.append([int(x) for x in tr if str(x) != 'nan'])
    return(newlists)

def create_XY_prefix(log, mappingsize, prefixlen):
    X = []
    Y = []
    for i in range(0, len(log)):
        for k in range(1, len(log[i])):
            X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
            y = [0] *(mappingsize)
            y[int(log[i][k])-1] = 1
            Y.append(y)        
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
    return(np.array(X), np.array(Y))

def get_model_onedirectional(maxlen, num_chars, embedding, n_layers, lstmsize, dropout, l1, l2):
    emb_size = math.ceil(num_chars**0.25)
    model = Sequential()
    if embedding == True: #if you use embedding layer input should be number encoded
        model.add(Input(shape=(maxlen, )))
        model.add(Embedding(input_dim=num_chars+1, output_dim=emb_size))
    else:
        model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
    model.add(LSTM(lstmsize,
                   kernel_initializer='glorot_uniform',
                   return_sequences=(n_layers != 1),
                   kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),
                   input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
        return_sequences = (i+1 != n_layers)
        model.add(LSTM(lstmsize,
                       kernel_initializer='glorot_uniform',
                       return_sequences=return_sequences,
                       kernel_regularizer=regularizers.l1_l2(l1,l2),
                       recurrent_regularizer=regularizers.l1_l2(l1,l2)))
        model.add(Dropout(dropout))
    
    model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
    opt = Adam(learning_rate=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    return model

def get_startend(log): #get first and last activity, we use it to stop the simulation. Should be changed!!!
    return log[0][0], log[0][-1]

def get_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    results = rnnmodel.predict(all_x)
    return results

def normalize(probs): #normalize probabilities to sum to 1
    examplesize, actsize = probs.shape
    newy = []
    for i in range(examplesize):
        normalizer = 1 / float( sum(probs[i]) )
        ynorm = [float(l) * normalizer for l in probs[i]]
        newy.append(ynorm)
    return newy


def choose_act_all(all_y): #randomly choose an activity, stochastically
    #p want a list of probabilities    
    chosen_acts = []
    for i in range(len(all_y)):
        chosen_acts.append(np.random.choice(np.arange(0, len(all_y[i])), p=all_y[i])+1)  
    return(chosen_acts)   # +1 because number encodig starts at 1 not 0

def cut_end(log, endact):
    logsize, tracesize = log.shape
    print(log.shape)
    newlog = []
    for i in range(0, logsize):
        trace = []
        for j in range(0, tracesize):
            if log[i][j] == endact:
                trace.append(log[i][j])
                break
            else:
                trace.append(log[i][j])
        newlog.append(trace)
    return(newlog)


def simulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = get_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 

def save_log(loglist, filename): #save a list of lists 
    df = pd.DataFrame.from_records(loglist)
    df.to_csv(filename, index=False)

def number_to_one_hot_X(X, dict_size): #if we want 
    newX = []
    for example in X:
        new_ex = []
        for i in range(len(example)):
            onehot = [0]*dict_size #changed
            if example[i] != 0:
                onehot[example[i] - 1] = 1 #-1 because 
            new_ex.append(onehot)
        newX.append(new_ex)
    return(np.array(newX))


def OHget_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    all_x = number_to_one_hot_X(all_x, nr_act)
    results = rnnmodel.predict(all_x)
    return results




def OHsimulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = OHget_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 






In [3]:
def fit_models(start_variant, end_variant):
    mappingfilename = 'mapping.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())
    for variant in range(start_variant, end_variant):
        filename = 'Sim_sets/log_'+ str(variant) + '.csv'
        if os.path.exists(filename):
            print("Already exists:", variant)
            continue
        trainname = 'Train_sets/log_'+ str(variant) + '.csv'
        train_log = remove_nan(import_log(trainname))

        maxLength = max(map(len, train_log))

        X_train, y_train = create_XY_prefix(train_log, len(mapping), 10)
        X_train = number_to_one_hot_X(X_train, len(mapping))

        model = get_model_onedirectional(maxlen=10, num_chars=len(mapping), embedding=False, n_layers=1, lstmsize=32, 
                                         dropout = 0.4, l1=0.001, l2=0.001)
        model.summary()
        batch_size = 128
        early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
        lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                       min_delta=0.0001, cooldown=0, min_lr=0)

        # train the model, output generated text after each iteration
        history = model.fit(X_train, y_train, 
                            validation_split=0.2,
                            callbacks=[early_stopping, lr_reducer],
                            batch_size=batch_size, epochs=600, verbose=2)


        #save RNN model in case we ever need it
        modelfilename = 'Models/log_'+ str(variant) + '.csv'
        model.save(modelfilename)

        start,end = get_startend(train_log)

        #CHANGE THIS BELOW!!!!!


        simlog = OHsimulate_log(model, 12000, start, end, maxLength*2, mapping, 10)

        #save simulated log
        SimLogName = 'Sim_sets/log_'+ str(variant) + '.csv'
        save_log(simlog, SimLogName)

In [4]:
fit_models(0, 28)

Already exists: 0
Already exists: 1
Already exists: 2
Already exists: 3
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                6272      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                528       
Total params: 6,800
Trainable params: 6,800
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
1535/1535 - 13s - loss: 0.5680 - val_loss: 0.3003
Epoch 2/600
1535/1535 - 8s - loss: 0.3112 - val_loss: 0.2795
Epoch 3/600
1535/1535 - 7s - loss: 0.2887 - val_loss: 0.2627
Epoch 4/600
1535/1535 - 7s - loss: 0.2765 - val_loss: 0.2594
Epoch 5/600
1535/1535 - 9s - loss: 0.2681 - val_loss: 0.2474
Epoch 

Epoch 120/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2095
Epoch 121/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2099
Epoch 122/600
1535/1535 - 7s - loss: 0.2154 - val_loss: 0.2103
Epoch 123/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2100
Epoch 124/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2093
Epoch 125/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2096
Epoch 126/600
1535/1535 - 7s - loss: 0.2151 - val_loss: 0.2092
Epoch 127/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2098
Epoch 128/600
1535/1535 - 7s - loss: 0.2151 - val_loss: 0.2098
Epoch 129/600
1535/1535 - 9s - loss: 0.2151 - val_loss: 0.2099
Epoch 130/600
1535/1535 - 8s - loss: 0.2151 - val_loss: 0.2096
Epoch 131/600
1535/1535 - 7s - loss: 0.2152 - val_loss: 0.2094
Epoch 132/600
1535/1535 - 6s - loss: 0.2150 - val_loss: 0.2098
Epoch 133/600
1535/1535 - 7s - loss: 0.2151 - val_loss: 0.2091
Epoch 134/600
1535/1535 - 6s - loss: 0.2151 - val_loss: 0.2101
Epoch 135/600
1535/1535 - 6s - loss: 0.2150 - val_loss:

Epoch 250/600
1535/1535 - 7s - loss: 0.2110 - val_loss: 0.2069
Epoch 251/600
1535/1535 - 7s - loss: 0.2110 - val_loss: 0.2069
Epoch 252/600
1535/1535 - 7s - loss: 0.2108 - val_loss: 0.2069
Epoch 253/600
1535/1535 - 7s - loss: 0.2108 - val_loss: 0.2069
Epoch 254/600
1535/1535 - 7s - loss: 0.2109 - val_loss: 0.2069
Epoch 255/600
1535/1535 - 7s - loss: 0.2109 - val_loss: 0.2069
Epoch 256/600
1535/1535 - 7s - loss: 0.2110 - val_loss: 0.2069
Epoch 257/600
1535/1535 - 8s - loss: 0.2108 - val_loss: 0.2069
Epoch 258/600
1535/1535 - 8s - loss: 0.2109 - val_loss: 0.2069
Epoch 259/600
1535/1535 - 7s - loss: 0.2108 - val_loss: 0.2069
Epoch 260/600
1535/1535 - 8s - loss: 0.2109 - val_loss: 0.2069
Epoch 261/600
1535/1535 - 8s - loss: 0.2109 - val_loss: 0.2069
Epoch 262/600
1535/1535 - 8s - loss: 0.2110 - val_loss: 0.2069
Epoch 263/600
1535/1535 - 8s - loss: 0.2108 - val_loss: 0.2069
Epoch 264/600
1535/1535 - 9s - loss: 0.2110 - val_loss: 0.2069
Epoch 265/600
1535/1535 - 10s - loss: 0.2110 - val_loss

INFO:tensorflow:Assets written to: Models/log_4.csv\assets


INFO:tensorflow:Assets written to: Models/log_4.csv\assets


...

INFO:tensorflow:Assets written to: Models/log_26.csv\assets


[[8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 ...
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]]
finding activity nr 2
[[8]
 [8]
 [8]
 ...
 [8]
 [8]
 [8]]
finding activity nr 3
[[ 8 15]
 [ 8 15]
 [ 8 15]
 ...
 [ 8 15]
 [ 8 15]
 [ 8 15]]
finding activity nr 4
[[ 8 15 14]
 [ 8 15 14]
 [ 8 15 14]
 ...
 [ 8 15 14]
 [ 8 15 14]
 [ 8 15 14]]
finding activity nr 5
[[ 8 15 14  7]
 [ 8 15 14  7]
 [ 8 15 14  7]
 ...
 [ 8 15 14  7]
 [ 8 15 14  7]
 [ 8 15 14  7]]
finding activity nr 6
[[ 8 15 14  7 11]
 [ 8 15 14  7 11]
 [ 8 15 14  7 11]
 ...
 [ 8 15 14  7 11]
 [ 8 15 14  7 11]
 [ 8 15 14  7 11]]
finding activity nr 7
[[ 8 15 14  7 11  7]
 [ 8 15 14  7 11  5]
 [ 8 15 14  7 11  5]
 ...
 [ 8 15 14  7 11  5]
 [ 8 15 14  7 11  5]
 [ 8 15 14  7 11  5]]
finding activity nr 8
[[ 8 15 14 ... 11  7 11]
 [ 8 15 14 ... 11  5  6]
 [ 8 15 14 ... 11  5  6]
 ...
 [ 8 15 14 ... 11  5  6]
 [ 8 15 14 ... 11  5  6]
 [ 8 15 14 ... 11  5  6]]
finding activity nr 9
[[ 8 15 14 ...  7 11  7]
 [ 8 15

finding activity nr 50
[[ 8 15 14 ...  3  3 15]
 [ 8 15 14 ...  3  4  1]
 [ 8 15 14 ...  7 11  7]
 ...
 [ 8 15 14 ...  3 15 14]
 [ 8 15 14 ...  1  4  1]
 [ 8 15 14 ... 11  7 11]]
finding activity nr 51
[[ 8 15 14 ...  3 15 14]
 [ 8 15 14 ...  4  1  4]
 [ 8 15 14 ... 11  7 11]
 ...
 [ 8 15 14 ... 15 14  7]
 [ 8 15 14 ...  4  1  4]
 [ 8 15 14 ...  7 11  5]]
finding activity nr 52
[[ 8 15 14 ... 15 14  7]
 [ 8 15 14 ...  1  4  1]
 [ 8 15 14 ...  7 11  7]
 ...
 [ 8 15 14 ... 14  7 11]
 [ 8 15 14 ...  1  4  1]
 [ 8 15 14 ... 11  5  6]]
finding activity nr 53
[[ 8 15 14 ... 14  7 11]
 [ 8 15 14 ...  4  1  4]
 [ 8 15 14 ... 11  7 11]
 ...
 [ 8 15 14 ...  7 11  7]
 [ 8 15 14 ...  4  1  4]
 [ 8 15 14 ...  5  6 10]]
finding activity nr 54
[[ 8 15 14 ...  7 11  5]
 [ 8 15 14 ...  1  4  1]
 [ 8 15 14 ...  7 11  5]
 ...
 [ 8 15 14 ... 11  7 11]
 [ 8 15 14 ...  1  4  1]
 [ 8 15 14 ...  6 10 12]]
finding activity nr 55
[[ 8 15 14 ... 11  5  6]
 [ 8 15 14 ...  4  1 13]
 [ 8 15 14 ... 11  5  6]
 ...
 [

finding activity nr 96
[[ 8 15 14 ... 12 10 12]
 [ 8 15 14 ...  5  6 10]
 [ 8 15 14 ...  7 11  5]
 ...
 [ 8 15 14 ...  2 16  4]
 [ 8 15 14 ... 16  4  1]
 [ 8 15 14 ... 11  5  6]]
finding activity nr 97
[[ 8 15 14 ... 10 12  2]
 [ 8 15 14 ...  6 10 12]
 [ 8 15 14 ... 11  5  6]
 ...
 [ 8 15 14 ... 16  4  1]
 [ 8 15 14 ...  4  1 13]
 [ 8 15 14 ...  5  6 10]]
finding activity nr 98
[[ 8 15 14 ... 12  2 16]
 [ 8 15 14 ... 10 12 10]
 [ 8 15 14 ...  5  6 10]
 ...
 [ 8 15 14 ...  4  1 13]
 [ 8 15 14 ...  1 13  9]
 [ 8 15 14 ...  6 10 12]]
finding activity nr 99
[[ 8 15 14 ...  2 16  4]
 [ 8 15 14 ... 12 10 12]
 [ 8 15 14 ...  6 10 12]
 ...
 [ 8 15 14 ...  1 13  9]
 [ 8 15 14 ... 13  9  3]
 [ 8 15 14 ... 10 12  2]]
finding activity nr 100
[[ 8 15 14 ... 16  4  1]
 [ 8 15 14 ... 10 12 10]
 [ 8 15 14 ... 10 12 10]
 ...
 [ 8 15 14 ... 13  9  3]
 [ 8 15 14 ...  9  3  3]
 [ 8 15 14 ... 12  2 16]]
finding activity nr 101
[[ 8 15 14 ...  4  1  4]
 [ 8 15 14 ... 12 10 12]
 [ 8 15 14 ... 12 10 12]
 ...


FileNotFoundError: [Errno 2] No such file or directory: 'Train_sets/log_27.csv'